# Qrack BQ

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

You also need the `cuQuantum` package, just for this notebook. [`NVIDIA cuQuantum`](https://docs.nvidia.com/cuda/cuquantum/latest/) provides APIs including state vector simulation and tensor network simulation. In this notebook, we'll show you how take a random or arbitrary circuit, simplify it with the `QuantumCircuit` class from Qrack, convert it into a Qiskit circuit in proper format to load as a tensor network with `CircuitToEinsum` in `cuQuantum`, then do whatever simulation and manipulation of the tensor network you wish with `cuQuantum`.

In [1]:
# How wide of an example circuit should we generate? (In qubits)
width = 6

In [2]:
import random
# import sys
import time

# For example, if your Jupyter installation uses pip:
# sys.executable} -m pip install pyqrack

# import os

# If simulating with Qrack, change these according to your OpenCL devices and system specifications.
# (To use just `QrackCircuit`, we don't need to consider these.)
# os.environ['QRACK_OCL_DEFAULT_DEVICE']='0'
# os.environ['QRACK_QUNITMULTI_DEVICES']='0'
# os.environ['QRACK_QPAGER_DEVICES']='0'
# os.environ['QRACK_QPAGER_DEVICES_HOST_POINTER']='0'
# os.environ['QRACK_MAX_PAGE_QB']='27'
# os.environ['QRACK_MAX_ALLOC_MB']='15872'.
# os.environ['QRACK_MAX_PAGING_QB']='30'
# os.environ['QRACK_MAX_CPU_QB']='32'
# os.environ['QRACK_QTENSORNETWORK_THRESHOLD_QB']='30'

All this next cell does is provide a function for generating random circuits, for our example that follows.

In [3]:
def random_circuit(width, sim):
    single_bit_gates = sim.h, sim.x, sim.y, sim.z, sim.s, sim.t 
    two_bit_gates = sim.mcx, sim.mcz
    all_bits = list(range(0, width))
    
    for i in range(width):
        # Single bit gates
        for j in range(width):
            gate = random.choice(single_bit_gates)
            gate(j)
            
        # Fully-connected couplers:
        ###########################
        unused_bits = random.sample(all_bits, width)
        while len(unused_bits) > 1:
            b1 = unused_bits.pop()
            b2 = unused_bits.pop()
            gate = random.choice(two_bit_gates)
            gate([b1], b2)

    sim.m_all()

[`QrackSimulator`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_simulator.py) is the "workhorse" of the `pyqrack` package. It instantiates simulated "registers" of qubits that we can act basic quantum gates between, to form arbitrary universal quantum circuits.

`QrackCircuit` is an optional class for optimizing compilation. With it, one can define a circuit in advance, which is optimized upon definition. Then, one can save the optimized result to a file and later load it into a new or existing `QrackCircuit` instance. Ultimately, the circuit is executed by calling `run()` on a `QrackCircuit`, with a parameter of `QrackSimulator` of appropriate size.

`QrackCircuit` can be converted to-and-from Qiskit's `QuantumCircuit` class, for ease of use. You'll notice that `QrackCircuit` needs the intermediary of disk input/output to do this, in cases: because `QrackCircuit` is ultimately at "C++ level," rather than "Python level," it can't directly do things like print circuit diagrams or list its internal gates. However, with disk I/O, it can rely on Qiskit for this, while `QrackCircuit` provides a fairly powerful way of simplifying input circuits and outputting fully-variational Qiskit circuits.

In [4]:
from pyqrack import QrackSimulator

start = time.perf_counter()
qsim = QrackSimulator(width)
random_circuit(width, qsim)
end = time.perf_counter()
print("Time: ", end - start)

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
Time:  0.08097886999894399


**(Happy Qracking! You rock!)**